This python code has been developped by [Taïs Grippa](https://github.com/tgrippa) (Université Libre de Bruxelles). 

Code developped on Ubuntu 22.04 (Ubuntu Jammy) and GRASS GIS 8.0.2 using the Docker environment [available here](https://github.com/tgrippa/Weaksupervision_Vaihingen).

# Define working environment

**Import libraries**

In [1]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile
import glob
import math
import pickle
print(sys.version)

3.10.4 (main, Jun 29 2022, 12:14:53) [GCC 11.2.0]


In [2]:
## Import multiprocessing and functools libraries
import multiprocessing
from multiprocessing import Pool
from functools import partial

**Add folder with SCR provided belong to this notebook**

In [3]:
# Add local module to the path
src = os.path.abspath(os.path.join(os.environ['HOME'],'github','SRC'))
if src not in sys.path:
    sys.path.append(src)

**Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev)**

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [5]:
exec(open(os.path.join(os.environ['HOME'],'github','SRC', 'config.py')).read())

In [6]:
print(config_parameters)

{'GISBASE': '/usr/lib/grass78', 'PYTHONLIB': '/usr/bin/python3', 'gisdb': '/home/tais/GRASSDATA', 'location': 'flair-one', 'permanent_mapset': 'PERMANENT', 'locationepsg': '2154', 'outputfolder': '/home/tais/result', 'inputdir': '/home/tais/data'}


In [7]:
print(data)

{'legend': '/home/tais/github/Legend.txt'}


In [8]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [9]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

PATH	= /.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/grass78/bin:/usr/lib/grass78/script:/usr/lib/grass78/lib 	
HOSTNAME	= 9e8ce31548c9 	
DISPLAY	= unixlocalhost:10.0 	
LANG	= C.UTF-8 	
LC_ALL	= C.UTF-8 	
JUPYTER_ENABLE_LAB	= yes 	
TINI_VERSION	= v0.6.0 	
HOME	= /home/tais 	
GIT_PYTHON_REFRESH	= quiet 	
JPY_PARENT_PID	= 7 	
TERM	= xterm-color 	
CLICOLOR	= 1 	
PAGER	= cat 	
GIT_PAGER	= cat 	
MPLBACKEND	= module://matplotlib_inline.backend_inline 	
PYTHONPATH	= :/usr/lib/grass78/etc/python:/usr/lib/grass78/etc/python/grass:/usr/lib/grass78/etc/python/grass/script 	
LD_LIBRARY_PATH	= :/usr/lib/grass78/lib 	
GISBASE	= /usr/lib/grass78 	
PYTHONLIB	= /usr/bin/python3 	
GIS_LOCK	= $$ 	
GISRC	= /home/tais/.grass7/rc 	


**GRASS GIS Python libraries**

In [10]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

**Other functions**

In [11]:
from grass.script import vector
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name
# Import function that check and create folder
from mkdir import check_create_dir
# Import function that check if GRASS GIS add-on is installed and install it if needed
from gextension import check_install_addon
# Import function for sorting string number naturally
from sorting_natural import natural_keys

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

In [12]:
def launch_mapset(mapset):
    #Declare empty list that will contain the messages to return
    return_message = []
    # Init
    gsetup.init(config_parameters['GISBASE'], config_parameters['gisdb'], config_parameters['location'], mapset)
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    return_message.append(check_location(config_parameters["gisdb"],config_parameters['location'],config_parameters["locationepsg"]))
    # Check if mapset exists
    return_message.append(check_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Change the current working GRASS GIS session mapset
    return_message.append(working_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Return
    return return_message

In [13]:
def import_image(image_path, quiet=True):
    if quiet:
        gscript.run_command('g.gisenv', set="DEBUG=-1")
    # Import
    gscript.run_command('r.in.gdal', quiet=True, overwrite=True, input=image_path, output="img")
    # Rename layers
    gscript.run_command('g.rename', quiet=True, overwrite=True, raster='img.1,red')
    gscript.run_command('g.rename', quiet=True, overwrite=True, raster='img.2,green')
    gscript.run_command('g.rename', quiet=True, overwrite=True, raster='img.3,blue')
    gscript.run_command('g.rename', quiet=True, overwrite=True, raster='img.4,nir')
    gscript.run_command('g.rename', quiet=True, overwrite=True, raster='img.5,ndsm')
    if quiet:
        gscript.run_command('g.gisenv', set="DEBUG=0")

In [54]:
def compute_ndvi():
    # Define computational region
    gscript.run_command('g.region', raster='red')
    # Use r.mapcalc to create new raster
    formula = "{new} = float({nir}-{red})/float({nir}+{red})"
    formula = formula.format(new='ndvi', nir='nir', red='red')
    #stdout = gscript.read_command('r.mapcalc', expression=formula, quiet=True, overwrite=True)
    gscript.mapcalc(formula, quiet=True, overwrite=True)
    gscript.run_command('r.null', map='ndvi', null='0')
    gscript.run_command('r.colors', quiet=True, map='ndvi', color='ndvi')

In [15]:
def compute_textures():
    # Define computational region
    gscript.run_command('g.region', raster='red')
    # Compute textures on Red band
    if 'text_red_ASM' not in gscript.list_strings('raster', 'text_red_*') or 'text_red_IDM' not in gscript.list_strings('raster', 'text_red_*'):
        gscript.run_command('r.texture', flags='n', overwrite=True, quiet=True, input='red', output='text_red', size='11', method='asm,idm')
        for x in gscript.list_strings('raster', pattern='text_red_*'):
            gscript.run_command('r.null', quiet=True, map=x, null='0')
            gscript.run_command('r.colors', flags='e', quiet=True, map=x, color='grey')  # Change color ramp to "grey" with histogram equalisation
    # Compute textures on Green band
    if 'text_green_DE' not in gscript.list_strings('raster', 'text_green_*') or 'text_green_Entr' not in gscript.list_strings('raster', 'text_green_*'):
        gscript.run_command('r.texture', flags='n', overwrite=True, quiet=True, input='green', output='text_green', size='11', method='de,entr')
        for x in gscript.list_strings('raster', pattern='text_green_*'):
            gscript.run_command('r.null', quiet=True, map=x, null='0')
            gscript.run_command('r.colors', flags='e', quiet=True, map=x, color='grey')
    # Compute textures on Nir band
    if 'text_nir_DE' not in gscript.list_strings('raster', 'text_nir_*'):
        gscript.run_command('r.texture', flags='n', overwrite=True, quiet=True, input='nir', output='text_nir', size='11', method='de')
        for x in gscript.list_strings('raster', pattern='text_nir_*'):
            gscript.run_command('r.null', quiet=True, map=x, null='0')
            gscript.run_command('r.colors', flags='e', quiet=True, map=x, color='grey')

In [24]:
def fix_textures():
    # Define computational region
    gscript.run_command('g.region', raster='red')
    # Fix issues with some textures layers having '-0' values (example: text_dsm_DE on Image 1 or 5)
    for x in gscript.list_pairs(type='rast',pattern="text_*"):
        if gscript.raster_info(x[0])['min']==-0.0:
            #print("Fixing values of raster %s"%x[0])
            gscript.mapcalc(exp="{rast}_tmp = if({rast}==-0,0,float({rast}))".format(rast=x[0]), overwrite=True)
            gscript.read_command('g.rename', quiet=True, overwrite=True, raster='{rast}_tmp,{rast}'.format(rast=x[0]))

In [17]:
def worker(job_tupple):
    # Launch a new mapset for this image
    launch_mapset(job_tupple[0])
    # Import images and rename
    import_image(job_tupple[1])
    # Compute ndvi
    compute_ndvi()
    # Create texture
    compute_textures()
    # Fix issue in textures
    fix_textures()

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Create new directories

In [18]:
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder'])

The folder '/home/tais/result' already exists


# Import and preprocessing data in parallel GRASSGIS mapsets

In [19]:
import fnmatch

def find_files(path, pattern):
    for root, dirs, files in os.walk(path):
        for file in fnmatch.filter(files, pattern):
            yield os.path.join(root, file)

In [20]:
# Get a list of path to train images
train_folder = os.path.join(config_parameters['inputdir'],'train')
train_tupple = [(os.path.split(x)[-1].split('.tif')[0],x) for x in find_files(train_folder, "IMG_*.tif")]
print(f'There are {len(train_tupple)} train images')

There are 2950 train images


In [ ]:
# Launch processes in parallel
start_parallel = start_processing()
ncores = 20
p = Pool(ncores)
output = p.map(worker, train_tupple[:])  # Launch the processes for as many items in the list (if function with a return, the returned results are ordered thanks to 'map' function)
p.close()
p.join()
# Print
print_processing_time(start_parallel, "Computation (on %s cores) achieved in "%(ncores))